In [2]:
import pandas as pd
import numpy as np
import transbigdata as tbd
import geopandas as gpd

In [3]:
#相关设置
import warnings
warnings.filterwarnings("ignore")

In [6]:
# 读取数据
data = pd.read_hdf('./raw_bike_data/nyc_bike_data_201901-201912.h5',key='weather_data')
data.head()

,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
datetime,,,,,,,,,
2019-01-01 01:08:00,47 °F,46 °F,97 °%,ENE,3 °mph,0 °mph,29.76 °in,0.1 °in,Rain
2019-01-01 01:51:00,47 °F,46 °F,97 °%,CALM,0 °mph,0 °mph,29.73 °in,0.1 °in,Light Rain
2019-01-01 02:31:00,47 °F,45 °F,93 °%,CALM,0 °mph,0 °mph,29.69 °in,0.0 °in,Light Rain
2019-01-01 02:47:00,48 °F,46 °F,93 °%,SSW,5 °mph,0 °mph,29.67 °in,0.0 °in,Cloudy
2019-01-01 02:51:00,50 °F,48 °F,93 °%,SSW,5 °mph,0 °mph,29.66 °in,0.0 °in,Cloudy


In [8]:
# 修改单位
def unit_unify(raw):
    #修改文本格式
    for i in raw.columns:
        raw.loc[:,i] = raw.loc[:,i].map(lambda x:x.replace('\xa0°',''))
    #修改公式单位
    #公式1 C = (F-32)/1.8
    wendu = ['Temperature','Dew Point']
    for i in wendu:
        raw.loc[:,i] = raw.loc[:,i].map(lambda x:x.replace('F',''))
        raw.loc[:,i] = raw.loc[:,i].astype(float)
        raw.loc[:,i] = (raw.loc[:,i]-32)/1.8
    #公式2 英里 = 1.609344千米
    sudu = ['Wind Speed','Wind Gust']
    for i in sudu:
        raw.loc[:,i] = raw.loc[:,i].map(lambda x:x.replace('mph',''))
        raw.loc[:,i] = raw.loc[:,i].astype(float)
        raw.loc[:,i] = raw.loc[:,i]*1.609344
    #公式3 英寸 = 25.4 毫米(mm)
    jiangshui = ['Pressure','Precip.']
    for i in jiangshui:
        raw.loc[:,i] = raw.loc[:,i].map(lambda x:x.replace('in',''))
        raw.loc[:,i] = raw.loc[:,i].astype(float)
        raw.loc[:,i] = raw.loc[:,i]*25.4
    # 修改%
        raw.loc[:,'Humidity'] = raw.loc[:,'Humidity'].astype(str)
        raw.loc[:,'Humidity'] = raw.loc[:,'Humidity'].map(lambda x:x.replace('%',''))
        raw.loc[:,'Humidity'] = raw.loc[:,'Humidity'].astype(float)
    return raw

data_unify = unit_unify(data.copy(deep=True))
data_unify


,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
datetime,,,,,,,,,
2019-07-01 00:51:00,21.666667,12.222222,55.0,NNW,19.312128,0.0,758.190,0.0,Fair
2019-07-01 01:51:00,21.111111,12.222222,57.0,NNW,16.093440,0.0,758.444,0.0,Fair
2019-07-01 02:51:00,20.555556,12.777778,61.0,NW,19.312128,0.0,758.698,0.0,Fair
2019-07-01 03:51:00,20.000000,13.333333,65.0,NW,20.921472,0.0,758.952,0.0,Fair
2019-07-01 04:51:00,20.000000,12.777778,63.0,NNW,14.484096,0.0,758.952,0.0,Fair
...,...,...,...,...,...,...,...,...,...
2019-09-30 19:51:00,19.444444,10.555556,57.0,ESE,12.874752,0.0,767.334,0.0,Cloudy
2019-09-30 20:51:00,18.888889,10.555556,59.0,SE,14.484096,0.0,767.334,0.0,Cloudy
2019-09-30 21:51:00,18.888889,11.111111,60.0,S,12.874752,0.0,767.334,0.0,Cloudy


In [9]:
# 聚合时间
def timeagg(x):
    if len(x) ==0:
        return np.nan
    else:
        return x[-1]

data_unify = data_unify.resample(rule='H',label='right').apply(timeagg)


In [16]:
# 生成起始和结束日期
start_date = '2019-7-1 01:00:00'
end_date = '2019-10-1'

# 生成时间索引
time_index = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='H'),columns=['Time'])
time_index.set_index('Time',inplace=True)
time_index

""
Time
2019-07-01 01:00:00
2019-07-01 02:00:00
2019-07-01 03:00:00
2019-07-01 04:00:00
2019-07-01 05:00:00
...
2019-09-30 20:00:00
2019-09-30 21:00:00
2019-09-30 22:00:00


In [20]:
weather_data_clean = time_index.join(data_unify,how='left').fillna(method='ffill').fillna(method='bfill')
weather_data_clean

,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
Time,,,,,,,,,
2019-07-01 01:00:00,21.666667,12.222222,55.0,NNW,19.312128,0.0,758.190,0.0,Fair
2019-07-01 02:00:00,21.111111,12.222222,57.0,NNW,16.093440,0.0,758.444,0.0,Fair
2019-07-01 03:00:00,20.555556,12.777778,61.0,NW,19.312128,0.0,758.698,0.0,Fair
2019-07-01 04:00:00,20.000000,13.333333,65.0,NW,20.921472,0.0,758.952,0.0,Fair
2019-07-01 05:00:00,20.000000,12.777778,63.0,NNW,14.484096,0.0,758.952,0.0,Fair
...,...,...,...,...,...,...,...,...,...
2019-09-30 20:00:00,19.444444,10.555556,57.0,ESE,12.874752,0.0,767.334,0.0,Cloudy
2019-09-30 21:00:00,18.888889,10.555556,59.0,SE,14.484096,0.0,767.334,0.0,Cloudy
2019-09-30 22:00:00,18.888889,11.111111,60.0,S,12.874752,0.0,767.334,0.0,Cloudy


In [21]:
#写入文件 
with pd.HDFStore('raw_bike_data/nyc_bike_data_201907-201909.h5','a') as h5:
    h5.put('weather_data_clean',weather_data_clean,format='table')

In [18]:
time_start = '201907'  #时间格式，必须全为数字
time_end = '201909'
h5_file_name = 'nyc_bike_data' + '_' + time_start + '-' + time_end + '.h5'  # h5 文件名称
h5_path = 'raw_bike_data/'  # 目标路径
h5_file = h5_path + h5_file_name
region_file = 'raw_bike_data/Manhattan.json'
accuracy = 750
data = pd.read_hdf(h5_file, key='bike_data')
def get_bounds(region_file, accuracy=500):
    #研究区域
    geo_bounds = gpd.read_file(region_file)
    #数据栅格化
    params = tbd.area_to_params(geo_bounds, accuracy=accuracy, method='rect')
    params['theta'] = 30
    return geo_bounds, params
geo_bounds, params = get_bounds(region_file, accuracy=accuracy)
#数据预处理
#剔除研究范围外的数据点
bike_data = tbd.clean_outofshape(data, geo_bounds, col=['slon', 'slat'])
bike_data = tbd.clean_outofshape(bike_data,
                                    geo_bounds,
                                    col=['elon', 'elat'])
print(len(bike_data) / len(data))
#剔除过长与过短的共享单车出行
bike_data['distance'] = tbd.getdistance(bike_data['slon'],
                                        bike_data['slat'],
                                        bike_data['elon'],
                                        bike_data['elat'])
#清洗骑行数据，删除过长与过短的出行（小于100，大于10000）
bike_data = bike_data[(bike_data['distance'] > 100)
                        & (bike_data['distance'] < 10000)]
bike_data['stime'] = pd.to_datetime(bike_data['stime'])
bike_data.set_index('stime', inplace=True)

0.7599637197734121


In [19]:
def timeagg(x):
    if len(x) ==0:
        return np.nan
    else:
        return x[-1]
bike_data.resample(rule='1H',label='right').apply(timeagg)

,etime,slat,slon,elat,elon,distance
stime,,,,,,
2019-07-01 01:00:00,2019-07-01 01:38:41.2920,40.750200,-73.990931,40.813358,-73.956461,7598.921594
2019-07-01 02:00:00,2019-07-01 02:02:22.8360,40.765005,-73.958185,40.761227,-73.960940,479.910954
2019-07-01 03:00:00,2019-07-01 03:18:23.9680,40.769155,-73.981918,40.798786,-73.952300,4132.101028
2019-07-01 04:00:00,2019-07-01 04:06:01.3100,40.788390,-73.974700,40.780578,-73.985624,1265.115845
2019-07-01 05:00:00,2019-07-01 05:31:06.8680,40.732219,-73.981656,40.789485,-73.952429,6826.998304
...,...,...,...,...,...,...
2019-09-30 20:00:00,2019-09-30 20:08:32.5760,40.784145,-73.983625,40.771497,-73.990460,1519.608777
2019-09-30 21:00:00,2019-09-30 21:14:57.0700,40.712690,-73.987763,40.727791,-73.985649,1688.559702
2019-09-30 22:00:00,2019-09-30 22:03:26.4780,40.741776,-74.001497,40.747348,-73.997236,716.114931
